## 1. Setup & Dependencies

In [1]:
# Install required packages
%pip install transformers torch pandas numpy plotly scikit-learn tqdm -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm
import warnings
import os
import json

warnings.filterwarnings('ignore')

# VADER Sentiment
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon', quiet=True)

print("="*80)
print("SENTIMENT ANALYSIS SETUP (VADER)")
print("="*80)
print("✅ NLTK VADER ready")
print(f"✅ All libraries loaded successfully!")

SENTIMENT ANALYSIS SETUP
✅ PyTorch version: 2.9.1+cpu
✅ Device: CPU
✅ All libraries loaded successfully!


## 2. Load Topic-Modeled Data

In [3]:
# Load the topic-modeled dataset
df = pd.read_csv('df_with_topics.csv')

# Convert date column
if 'at' in df.columns:
    df['at'] = pd.to_datetime(df['at'])

print("="*80)
print("DATA LOADED SUCCESSFULLY")
print("="*80)
print(f"Total Reviews: {len(df):,}")
print(f"\nRating Distribution:")
print(df['score'].value_counts().sort_index())
print(f"\nColumns: {list(df.columns)}")

# Check for topic columns
topic_cols = [col for col in df.columns if 'topic' in col.lower()]
print(f"\nTopic-related columns: {topic_cols}")

DATA LOADED SUCCESSFULLY
Total Reviews: 82,255

Rating Distribution:
score
1    14913
2     2369
3     3542
4    10036
5    51395
Name: count, dtype: int64

Columns: ['reviewId', 'userName', 'userImage', 'content', 'score', 'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion', 'text_length', 'word_count', 'processed_text', 'dominant_topic', 'topic_probability', 'sentiment_group', 'Topic_Label']

Topic-related columns: ['dominant_topic', 'topic_probability', 'Topic_Label']


## 3. Prepare Full Dataset for Analysis
**Strategy:** Analyze ALL reviews (100k) for comprehensive insights

In [4]:
# Use full dataset for sentiment analysis
print("="*80)
print("PREPARING FULL DATASET FOR SENTIMENT ANALYSIS")
print("="*80)

# Option 1: Analyze ALL reviews (recommended for comprehensive insights)
USE_FULL_DATASET = True  # Set to False for faster testing with sample

if USE_FULL_DATASET:
    analysis_df = df.copy()
    print(f"📊 Using FULL DATASET: {len(analysis_df):,} reviews")
    print(f"   Estimated processing time: ~30-45 minutes (depending on CPU/GPU)")
else:
    # Option 2: Use sample for faster testing
    one_star = df[df['score'] == 1].copy()
    five_star = df[df['score'] == 5].copy()
    
    SAMPLE_SIZE = 5000
    RANDOM_STATE = 42
    
    five_star_sample = five_star.sample(n=min(SAMPLE_SIZE, len(five_star)), random_state=RANDOM_STATE)
    analysis_df = pd.concat([one_star, five_star_sample], ignore_index=True)
    analysis_df = analysis_df.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)
    
    print(f"📊 Using SAMPLE: {len(analysis_df):,} reviews")
    print(f"   (1-star: {len(one_star):,}, 5-star sample: {len(five_star_sample):,})")
    print(f"   Estimated processing time: ~5-10 minutes")

print(f"\n✅ Analysis dataset ready!")
print(f"\nRating Distribution:")
print(analysis_df['score'].value_counts().sort_index())

print(f"\n💡 TIP: Processing will show a progress bar.")
print(f"   You can continue working while it runs in the background.")

PREPARING FULL DATASET FOR SENTIMENT ANALYSIS
📊 Using FULL DATASET: 82,255 reviews
   Estimated processing time: ~30-45 minutes (depending on CPU/GPU)

✅ Analysis dataset ready!

Rating Distribution:
score
1    14913
2     2369
3     3542
4    10036
5    51395
Name: count, dtype: int64

💡 TIP: Processing will show a progress bar.
   You can continue working while it runs in the background.


## 4. Initialize Sentiment Analysis Pipeline
**Model:** VADER (rule-based, fast)

In [9]:
# Initialize VADER sentiment analyzer
print("="*80)
print("LOADING VADER SENTIMENT ANALYZER")
print("="*80)

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

print("\n✅ VADER initialized successfully!")

# Quick test
sample_text = "This app is amazing!"
sample_score = sia.polarity_scores(sample_text)['compound']
label = 'Positive' if sample_score >= 0.05 else 'Negative' if sample_score <= -0.05 else 'Neutral'
print(f"\n🧪 Test inference: '{sample_text}' → {label} (compound={sample_score:.3f})")

LOADING VADER SENTIMENT ANALYZER

✅ VADER initialized successfully!

🧪 Test inference: 'This app is amazing!' → Positive (compound=0.624)


## 5. Batch Sentiment Inference
**Efficient processing with progress tracking**

In [10]:
def run_sentiment_analysis(texts, pos_thresh=0.05, neg_thresh=-0.05):
    """
    Run VADER sentiment on a list of texts.
    Returns labels and compound scores.
    """
    all_labels = []
    all_scores = []

    # Clean texts (handle NaN and empty strings)
    texts = [str(t) if pd.notna(t) and str(t).strip() else "no content" for t in texts]

    print(f"\n🔄 Processing {len(texts):,} reviews with VADER (no batching needed)...")

    for t in tqdm(texts, desc="Analyzing sentiment"):
        scores = sia.polarity_scores(t)
        comp = scores['compound']
        if comp >= pos_thresh:
            label = 'Positive'
        elif comp <= neg_thresh:
            label = 'Negative'
        else:
            label = 'Neutral'
        all_labels.append(label)
        all_scores.append(comp)

    return all_labels, all_scores

# Run sentiment analysis
print("="*80)
print("RUNNING SENTIMENT ANALYSIS (VADER)")
print("="*80)

texts = analysis_df['content'].tolist()

sentiment_labels, sentiment_scores = run_sentiment_analysis(texts)

# Add results to dataframe
analysis_df['ai_sentiment'] = sentiment_labels
analysis_df['ai_confidence'] = np.abs(sentiment_scores)  # use |compound| as confidence proxy
analysis_df['ai_compound'] = sentiment_scores

print(f"\n✅ Sentiment analysis complete!")
print(f"\nAI Sentiment Distribution:")
print(analysis_df['ai_sentiment'].value_counts())

RUNNING SENTIMENT ANALYSIS (VADER)

🔄 Processing 82,255 reviews with VADER (no batching needed)...


Analyzing sentiment: 100%|██████████| 82255/82255 [00:06<00:00, 13228.31it/s]



✅ Sentiment analysis complete!

AI Sentiment Distribution:
ai_sentiment
Positive    60116
Neutral     11303
Negative    10836
Name: count, dtype: int64


## 6. Create Ground Truth Labels
**Map star ratings to sentiment categories for validation**

In [11]:
# Map star ratings to sentiment
def map_rating_to_sentiment(score):
    """Map star ratings to sentiment categories"""
    if score <= 2:
        return 'Negative'
    elif score == 3:
        return 'Neutral'
    else:
        return 'Positive'

analysis_df['rating_sentiment'] = analysis_df['score'].apply(map_rating_to_sentiment)

print("="*80)
print("GROUND TRUTH LABELS CREATED")
print("="*80)
print("\nRating → Sentiment Mapping:")
print("  1-2 stars → Negative")
print("  3 stars   → Neutral")
print("  4-5 stars → Positive")
print("\nGround Truth Distribution:")
print(analysis_df['rating_sentiment'].value_counts())

GROUND TRUTH LABELS CREATED

Rating → Sentiment Mapping:
  1-2 stars → Negative
  3 stars   → Neutral
  4-5 stars → Positive

Ground Truth Distribution:
rating_sentiment
Positive    61431
Negative    17282
Neutral      3542
Name: count, dtype: int64


## 6A. Hybrid Sentiment (VADER + BERT on uncertain rows)
**Goal:** Keep VADER for confident rows, run BERT only on uncertain/selected rows, and capture agreement diagnostics.

In [12]:
# Hybrid sentiment: VADER baseline + BERT refinement on uncertain/selected rows
print("="*80)
print("HYBRID SENTIMENT (VADER base, BERT on uncertain rows)")
print("="*80)

# Default to VADER outputs
analysis_df['hybrid_sentiment'] = analysis_df['ai_sentiment']
analysis_df['hybrid_confidence'] = analysis_df['ai_confidence']
analysis_df['hybrid_source'] = 'VADER'

# Configuration for refinement
CONFIDENCE_MAX = 0.60        # run BERT when |compound| is below this
INCLUDE_NEUTRAL = True        # always refine Neutral rows
MANUAL_INDEXES = []           # optionally add row indices to force BERT (e.g., [123, 456])
BATCH_SIZE = 32

candidate_mask = analysis_df['ai_confidence'] <= CONFIDENCE_MAX
if INCLUDE_NEUTRAL:
    candidate_mask |= analysis_df['ai_sentiment'] == 'Neutral'
if MANUAL_INDEXES:
    candidate_mask |= analysis_df.index.isin(MANUAL_INDEXES)

candidates = analysis_df[candidate_mask].copy()
print(f"Candidates for BERT refinement: {len(candidates):,} rows ({len(candidates)/len(analysis_df)*100:.1f}%)")

if len(candidates) > 0:
    from transformers import pipeline

    bert_classifier = pipeline(
        'text-classification',
        model='distilbert-base-uncased-finetuned-sst-2-english',
        device=-1  # CPU; change to 0 to force GPU if available
    )

    bert_preds = []
    for start in range(0, len(candidates), BATCH_SIZE):
        batch_texts = (
            candidates['content']
            .iloc[start:start + BATCH_SIZE]
            .fillna('no content')
            .astype(str)
            .tolist()
        )
        bert_preds.extend(bert_classifier(batch_texts))

    bert_labels = ['Positive' if p['label'].upper().startswith('POS') else 'Negative' for p in bert_preds]
    bert_scores = [float(p['score']) for p in bert_preds]

    candidates.loc[:, 'hybrid_sentiment'] = bert_labels
    candidates.loc[:, 'hybrid_confidence'] = bert_scores
    candidates.loc[:, 'hybrid_source'] = 'BERT'

    # Merge back
    analysis_df.loc[candidates.index, ['hybrid_sentiment', 'hybrid_confidence', 'hybrid_source']] = (
        candidates[['hybrid_sentiment', 'hybrid_confidence', 'hybrid_source']]
    )
else:
    print("No rows met the refinement criteria; hybrid == VADER outputs.")

print("\nSource distribution (VADER vs BERT):")
print(analysis_df['hybrid_source'].value_counts())

print("\nAgreement: Hybrid vs VADER")
hybrid_vader_agree = (analysis_df['hybrid_sentiment'] == analysis_df['ai_sentiment']).mean()
print(f"  Agreement rate: {hybrid_vader_agree:.2%}")
print(f"  Rows changed by BERT: {(~(analysis_df['hybrid_sentiment'] == analysis_df['ai_sentiment'])).sum():,}")

print("\nRating alignment (ground truth):")
hybrid_accuracy = accuracy_score(analysis_df['rating_sentiment'], analysis_df['hybrid_sentiment'])
vader_accuracy = accuracy_score(analysis_df['rating_sentiment'], analysis_df['ai_sentiment'])
print(f"  Hybrid accuracy: {hybrid_accuracy:.2%}")
print(f"  VADER-only accuracy: {vader_accuracy:.2%}")

print("\n✅ Hybrid sentiment complete. Added columns: hybrid_sentiment, hybrid_confidence, hybrid_source")

HYBRID SENTIMENT (VADER base, BERT on uncertain rows)
Candidates for BERT refinement: 67,421 rows (82.0%)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu



Source distribution (VADER vs BERT):
hybrid_source
BERT     67421
VADER    14834
Name: count, dtype: int64

Agreement: Hybrid vs VADER
  Agreement rate: 81.64%
  Rows changed by BERT: 15,100

Rating alignment (ground truth):
  Hybrid accuracy: 88.60%
  VADER-only accuracy: 79.24%

✅ Hybrid sentiment complete. Added columns: hybrid_sentiment, hybrid_confidence, hybrid_source


## 7. Model Performance Report
**Comparing AI sentiment predictions against actual user ratings**

In [13]:
# Calculate performance metrics
y_true = analysis_df['rating_sentiment']
y_pred = analysis_df['ai_sentiment']

# Accuracy
accuracy = accuracy_score(y_true, y_pred)

# Classification report
class_report = classification_report(y_true, y_pred, output_dict=True)

print("="*80)
print("MODEL PERFORMANCE REPORT")
print("="*80)
print(f"\n📊 OVERALL ACCURACY: {accuracy:.2%}")
print("\n" + "="*80)
print("CLASSIFICATION METRICS BY SENTIMENT")
print("="*80)
print(classification_report(y_true, y_pred))

# Agreement analysis
analysis_df['sentiment_match'] = analysis_df['ai_sentiment'] == analysis_df['rating_sentiment']
match_rate = analysis_df['sentiment_match'].mean()

print(f"\n🎯 AGREEMENT RATE: {match_rate:.2%}")
print(f"   Reviews where AI agrees with user rating: {analysis_df['sentiment_match'].sum():,}")
print(f"   Reviews where AI disagrees: {(~analysis_df['sentiment_match']).sum():,}")

MODEL PERFORMANCE REPORT

📊 OVERALL ACCURACY: 79.24%

CLASSIFICATION METRICS BY SENTIMENT
              precision    recall  f1-score   support

    Negative       0.86      0.54      0.66     17282
     Neutral       0.07      0.23      0.11      3542
    Positive       0.92      0.90      0.91     61431

    accuracy                           0.79     82255
   macro avg       0.62      0.56      0.56     82255
weighted avg       0.87      0.79      0.82     82255


🎯 AGREEMENT RATE: 79.24%
   Reviews where AI agrees with user rating: 65,179
   Reviews where AI disagrees: 17,076


## 8. Confusion Matrix Visualization (Plotly)

In [14]:
# Create confusion matrix
labels = ['Negative', 'Neutral', 'Positive']
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Calculate percentages
cm_pct = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Create annotation text with counts and percentages
annotations = []
for i in range(len(labels)):
    for j in range(len(labels)):
        annotations.append(f"{cm[i,j]:,}\n({cm_pct[i,j]:.1f}%)")

annotations = np.array(annotations).reshape(len(labels), len(labels))

# Create Plotly heatmap
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=[f'Predicted: {l}' for l in labels],
    y=[f'Actual: {l}' for l in labels],
    colorscale='Blues',
    text=annotations,
    texttemplate='%{text}',
    textfont={'size': 14},
    hovertemplate='Actual: %{y}<br>Predicted: %{x}<br>Count: %{z}<extra></extra>',
    colorbar=dict(title='Count')
))

fig.update_layout(
    title=dict(
        text=f'<b>Confusion Matrix: AI Sentiment vs User Ratings</b><br>' +
             f'<sub>Overall Accuracy: {accuracy:.2%} | Agreement Rate: {match_rate:.2%}</sub>',
        x=0.5,
        font=dict(size=16)
    ),
    xaxis_title='AI Predicted Sentiment',
    yaxis_title='Actual Sentiment (from Rating)',
    height=500,
    width=700,
    template='plotly_white'
)

fig.show()

# Save confusion matrix data for dashboard
cm_data = {
    'confusion_matrix': cm.tolist(),
    'labels': labels,
    'accuracy': accuracy,
    'match_rate': match_rate,
    'classification_report': class_report
}

with open('confusion_matrix_data.json', 'w') as f:
    json.dump(cm_data, f, indent=2)

print("\n✅ Confusion matrix saved to 'confusion_matrix_data.json'")


✅ Confusion matrix saved to 'confusion_matrix_data.json'


## 9. Detailed Disagreement Analysis
**Understanding where AI and users disagree**

In [15]:
# Analyze disagreements
disagreements = analysis_df[~analysis_df['sentiment_match']].copy()

print("="*80)
print("DISAGREEMENT ANALYSIS")
print("="*80)
print(f"\nTotal disagreements: {len(disagreements):,} ({len(disagreements)/len(analysis_df)*100:.1f}%)")

# Cross-tabulation of disagreements
print("\nDisagreement Patterns:")
crosstab = pd.crosstab(
    disagreements['rating_sentiment'], 
    disagreements['ai_sentiment'],
    margins=True
)
print(crosstab)

# Sample disagreements
print("\n" + "="*80)
print("SAMPLE DISAGREEMENTS (for manual review)")
print("="*80)

# Show cases where 1-star was classified as Positive
false_positives = disagreements[
    (disagreements['rating_sentiment'] == 'Negative') & 
    (disagreements['ai_sentiment'] == 'Positive')
]

if len(false_positives) > 0:
    print("\n🔴 1-Star reviews classified as POSITIVE by AI:")
    for idx, row in false_positives.head(3).iterrows():
        print(f"\n  Rating: {row['score']} ⭐ | AI: {row['ai_sentiment']} ({row['ai_confidence']:.2f})")
        print(f"  Text: {str(row['content'])[:200]}...")

# Show cases where 5-star was classified as Negative
false_negatives = disagreements[
    (disagreements['rating_sentiment'] == 'Positive') & 
    (disagreements['ai_sentiment'] == 'Negative')
]

if len(false_negatives) > 0:
    print("\n🟢 5-Star reviews classified as NEGATIVE by AI:")
    for idx, row in false_negatives.head(3).iterrows():
        print(f"\n  Rating: {row['score']} ⭐ | AI: {row['ai_sentiment']} ({row['ai_confidence']:.2f})")
        print(f"  Text: {str(row['content'])[:200]}...")

DISAGREEMENT ANALYSIS

Total disagreements: 17,076 (20.8%)

Disagreement Patterns:
ai_sentiment      Negative  Neutral  Positive    All
rating_sentiment                                    
Negative                 0     4877      3083   7960
Neutral                712        0      2003   2715
Positive               802     5599         0   6401
All                   1514    10476      5086  17076

SAMPLE DISAGREEMENTS (for manual review)

🔴 1-Star reviews classified as POSITIVE by AI:

  Rating: 1 ⭐ | AI: Positive (0.27)
  Text: app ui not properly work why account create its this. not processed another step. how many times OTP genrets...

  Rating: 2 ⭐ | AI: Positive (0.68)
  Text: it will be an achievement if I am able to book a cylinder in one go. It needs tons of improvement and optimization...

  Rating: 1 ⭐ | AI: Positive (0.15)
  Text: convenience fee shouldn't be charged....

🟢 5-Star reviews classified as NEGATIVE by AI:

  Rating: 5 ⭐ | AI: Negative (0.40)
  Text: online log

## 10. Enrich Full Dataset & Export
**Apply sentiment to all reviews and save final dataset**

In [16]:
# For efficiency, we'll apply sentiment to remaining reviews
# Or use the analysis_df directly if full inference isn't needed

print("="*80)
print("PREPARING FINAL ENRICHED DATASET")
print("="*80)

# Add time-based features for dashboard
if 'at' in analysis_df.columns:
    analysis_df['month_year'] = analysis_df['at'].dt.to_period('M').astype(str)
    analysis_df['year'] = analysis_df['at'].dt.year
    analysis_df['month'] = analysis_df['at'].dt.month
    analysis_df['month_name'] = analysis_df['at'].dt.month_name()

# Ensure topic label column exists
if 'dominant_topic' in analysis_df.columns and 'Topic_Label' not in analysis_df.columns:
    analysis_df['Topic_Label'] = 'Topic ' + (analysis_df['dominant_topic'] + 1).astype(str)

# Create sentiment score (-1 to 1 scale)
def calculate_sentiment_score(row):
    if row['ai_sentiment'] == 'Positive':
        return row['ai_confidence']
    elif row['ai_sentiment'] == 'Negative':
        return -row['ai_confidence']
    else:
        return 0

analysis_df['sentiment_score'] = analysis_df.apply(calculate_sentiment_score, axis=1)

print(f"\nFinal dataset shape: {analysis_df.shape}")
print(f"\nColumns in final dataset:")
for col in analysis_df.columns:
    print(f"  • {col}")

PREPARING FINAL ENRICHED DATASET

Final dataset shape: (82255, 31)

Columns in final dataset:
  • reviewId
  • userName
  • userImage
  • content
  • score
  • thumbsUpCount
  • reviewCreatedVersion
  • at
  • replyContent
  • repliedAt
  • appVersion
  • text_length
  • word_count
  • processed_text
  • dominant_topic
  • topic_probability
  • sentiment_group
  • Topic_Label
  • ai_sentiment
  • ai_confidence
  • ai_compound
  • rating_sentiment
  • hybrid_sentiment
  • hybrid_confidence
  • hybrid_source
  • sentiment_match
  • month_year
  • year
  • month
  • month_name
  • sentiment_score


In [17]:
# Export final enriched dataset
OUTPUT_FILE = 'df_final_enriched.csv'

analysis_df.to_csv(OUTPUT_FILE, index=False)

print("="*80)
print("EXPORT COMPLETE")
print("="*80)
print(f"\n✅ Saved: {OUTPUT_FILE}")
print(f"   Rows: {len(analysis_df):,}")
print(f"   Columns: {len(analysis_df.columns)}")
print(f"\n📊 Dataset includes:")
print(f"   • Original review data (content, score, appVersion, date)")
print(f"   • Topic assignments (from LDA model)")
print(f"   • AI sentiment labels (Negative/Neutral/Positive)")
print(f"   • AI confidence scores")
print(f"   • Sentiment scores (-1 to 1)")
print(f"   • Time features (month_year, year, month)")
print(f"\n🎯 Ready for dashboard visualization!")

EXPORT COMPLETE

✅ Saved: df_final_enriched.csv
   Rows: 82,255
   Columns: 31

📊 Dataset includes:
   • Original review data (content, score, appVersion, date)
   • Topic assignments (from LDA model)
   • AI sentiment labels (Negative/Neutral/Positive)
   • AI confidence scores
   • Sentiment scores (-1 to 1)
   • Time features (month_year, year, month)

🎯 Ready for dashboard visualization!


## 11. Summary Statistics for Dashboard

In [18]:
# Generate summary statistics
print("="*80)
print("SUMMARY STATISTICS")
print("="*80)

# Global sentiment health
global_sentiment = analysis_df['sentiment_score'].mean()
print(f"\n🌡️ GLOBAL SENTIMENT HEALTH: {global_sentiment:.3f}")
print(f"   Scale: -1 (Very Negative) to +1 (Very Positive)")
print(f"   Interpretation: {'Positive' if global_sentiment > 0 else 'Negative'} overall sentiment")

# Sentiment by rating
print("\n📊 AI Sentiment by Star Rating:")
sentiment_by_rating = analysis_df.groupby('score')['ai_sentiment'].value_counts(normalize=True).unstack().fillna(0)
print(sentiment_by_rating.round(3) * 100)

# Topic distribution
if 'Topic_Label' in analysis_df.columns:
    print("\n📌 Topic Distribution:")
    print(analysis_df['Topic_Label'].value_counts())

print("\n" + "="*80)
print("✅ SENTIMENT ANALYSIS COMPLETE")
print("="*80)
print("\n🚀 Next Step: Run Notebook 03 to launch the interactive dashboard!")

SUMMARY STATISTICS

🌡️ GLOBAL SENTIMENT HEALTH: 0.300
   Scale: -1 (Very Negative) to +1 (Very Positive)
   Interpretation: Positive overall sentiment

📊 AI Sentiment by Star Rating:
ai_sentiment  Negative  Neutral  Positive
score                                    
1                 55.6     28.0      16.4
2                 43.5     29.6      26.9
3                 20.1     23.3      56.5
4                  3.5     10.2      86.3
5                  0.9      8.9      90.2

📌 Topic Distribution:
Topic_Label
Topic 2.0    22530
Topic 4.0    21114
Topic 1.0    20417
Topic 3.0    18194
Name: count, dtype: int64

✅ SENTIMENT ANALYSIS COMPLETE

🚀 Next Step: Run Notebook 03 to launch the interactive dashboard!
